In [1]:
path = 'E:\SSDA2019\Data'

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
os.path.isdir(path)

True

In [4]:
train = pd.read_csv(os.path.join(path, 'Train.csv'), header=None)
train.columns = ['img_id', 'label']
train.label = train.label - 1
train.head()

,img_id,label
0,1,0
1,10,0
2,11,0
3,12,0
4,13,0


In [5]:
val = pd.read_csv(os.path.join(path, 'Validation.csv'), header=None)
val.columns = ['img_id', 'label']
val.label = val.label - 1
val.head()

,img_id,label
0,41,0
1,42,0
2,43,0
3,44,0
4,45,0


In [6]:
np.unique(val.label, return_counts=True)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [7]:
from keras.utils import to_categorical
y_train = to_categorical(train.label)

Using TensorFlow backend.


In [8]:
y_val = to_categorical(val.label)

In [9]:
from keras.preprocessing.image import img_to_array
import cv2
from tqdm import tqdm

In [10]:
train_path = os.path.join(path, 'Train')
image_data = []
for imagePath in tqdm(train.img_id.values):
    image = cv2.imread(os.path.join(train_path,str(imagePath)+'.png'), 0)
    image = cv2.resize(image, (32, 32))    #Uncomment if required....
    image = img_to_array(image)
    image_data.append(image/255.0)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:49<00:00, 202.45it/s]


In [11]:
val_path = os.path.join(path, 'Validation')
image_data_val = []
for imagePath in tqdm(val.img_id.values):
    image = cv2.imread(os.path.join(val_path,str(imagePath)+'.png'), 0)
    image = cv2.resize(image, (32, 32))    #Uncomment if required....
    image = img_to_array(image)
    image_data_val.append(image/255.0)

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:27<00:00, 185.16it/s]


In [12]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(32,32,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Conv2D(16, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(250, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 16)        4624      
_________________________________________________________________
flatten_1 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               2704250   
Total params: 2,727,978
Trainable params: 2,727,978
Non-trainable params: 0
_________________________________________________________________


In [14]:
image_data = np.array(image_data)
image_data_val = np.array(image_data_val)

In [15]:
from keras.optimizers import Nadam

In [16]:
model.compile(optimizer=Nadam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.fit(image_data, y_train, epochs=10, verbose=1, validation_data=(image_data_val, y_val))

Instructions for updating:
Use tf.cast instead.
Train on 10000 samples, validate on 5000 samples
Epoch 1/10


 6752/10000 [===================>..........] - ETA: 13:52 - loss: 5.5237 - acc: 0.0000e+ - ETA: 7:27 - loss: 6.3931 - acc: 0.0156     - ETA: 5:16 - loss: 6.1371 - acc: 0.010 - ETA: 4:10 - loss: 6.0045 - acc: 0.007 - ETA: 3:30 - loss: 5.9062 - acc: 0.012 - ETA: 3:04 - loss: 5.8566 - acc: 0.010 - ETA: 2:45 - loss: 5.8099 - acc: 0.008 - ETA: 2:30 - loss: 5.7926 - acc: 0.007 - ETA: 2:19 - loss: 5.7637 - acc: 0.006 - ETA: 2:10 - loss: 5.7386 - acc: 0.012 - ETA: 2:03 - loss: 5.7168 - acc: 0.011 - ETA: 1:56 - loss: 5.6997 - acc: 0.010 - ETA: 1:51 - loss: 5.6884 - acc: 0.009 - ETA: 1:46 - loss: 5.6788 - acc: 0.008 - ETA: 1:42 - loss: 5.6683 - acc: 0.008 - ETA: 1:38 - loss: 5.6593 - acc: 0.007 - ETA: 1:38 - loss: 5.6508 - acc: 0.009 - ETA: 1:37 - loss: 5.6485 - acc: 0.008 - ETA: 1:35 - loss: 5.6428 - acc: 0.008 - ETA: 1:36 - loss: 5.6367 - acc: 0.007 - ETA: 1:36 - loss: 5.6312 - acc: 0.007 - ETA: 1:34 - loss: 5.6268 - acc: 0.007 - ETA: 1:32 - loss: 5.6244 - acc: 0.006 - ETA: 1:31 - loss: 5.6201

 6464/10000 [==================>...........] - ETA: 58s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 57s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 56s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 55s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 56s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 56s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 55s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 55s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 55s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 54s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 54s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 54s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 54s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 54s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 54s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 54s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 53s - loss: 16.0588 - acc: 0.0037    - ETA: 53s - loss: 16.0621 - acc: 0.003 - ETA: 53s - loss: 16.0651 - acc: 0.003 - ETA: 53s - loss: 16.0677 - acc: 0.003 - ETA: 53s - loss: 16.0701 - acc: 0.003 - ETA: 52s - loss: 16.0723 - acc: 0.002 - ETA: 

10000/10000 [==============================] - ETA: 20s - loss: 16.0511 - acc: 0.004 - ETA: 20s - loss: 16.0514 - acc: 0.004 - ETA: 20s - loss: 16.0518 - acc: 0.004 - ETA: 20s - loss: 16.0521 - acc: 0.004 - ETA: 20s - loss: 16.0500 - acc: 0.004 - ETA: 19s - loss: 16.0503 - acc: 0.004 - ETA: 19s - loss: 16.0506 - acc: 0.004 - ETA: 19s - loss: 16.0509 - acc: 0.004 - ETA: 19s - loss: 16.0513 - acc: 0.004 - ETA: 19s - loss: 16.0492 - acc: 0.004 - ETA: 18s - loss: 16.0495 - acc: 0.004 - ETA: 18s - loss: 16.0498 - acc: 0.004 - ETA: 18s - loss: 16.0502 - acc: 0.004 - ETA: 18s - loss: 16.0505 - acc: 0.004 - ETA: 18s - loss: 16.0508 - acc: 0.004 - ETA: 17s - loss: 16.0511 - acc: 0.004 - ETA: 17s - loss: 16.0514 - acc: 0.004 - ETA: 17s - loss: 16.0517 - acc: 0.004 - ETA: 17s - loss: 16.0520 - acc: 0.004 - ETA: 17s - loss: 16.0523 - acc: 0.004 - ETA: 16s - loss: 16.0526 - acc: 0.004 - ETA: 16s - loss: 16.0529 - acc: 0.004 - ETA: 16s - loss: 16.0509 - acc: 0.004 - ETA: 16s - loss: 16.0512 - acc: 0

 6496/10000 [==================>...........] - ETA: 38s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 41s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 40s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 40s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 41s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 42s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 41s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 41s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 41s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 41s - loss: 16.0677 - acc: 0.0031    - ETA: 41s - loss: 16.0723 - acc: 0.002 - ETA: 41s - loss: 16.0761 - acc: 0.002 - ETA: 41s - loss: 16.0406 - acc: 0.004 - ETA: 40s - loss: 16.0461 - acc: 0.004 - ETA: 41s - loss: 16.0509 - acc: 0.004 - ETA: 40s - loss: 16.0237 - acc: 0.005 - ETA: 40s - loss: 16.0292 - acc: 0.005 - ETA: 40s - loss: 16.0341 - acc: 0.005 - ETA: 40s - loss: 16.0386 - acc: 0.004 - ETA: 40s - loss: 16.0425 - acc: 0.004 - ETA: 40s - loss: 16.0461 - acc: 0.004 - ETA: 40s - loss: 16.0494 - acc: 0.004 - ETA: 40s - loss: 16.0524 - acc: 0

...

 6528/10000 [==================>...........] - ETA: 38s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 38s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 40s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 39s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 39s - loss: 16.0174 - acc: 0.0063    - ETA: 39s - loss: 16.0341 - acc: 0.005 - ETA: 39s - loss: 16.0461 - acc: 0.004 - ETA: 39s - loss: 16.0551 - acc: 0.003 - ETA: 40s - loss: 16.0621 - acc: 0.003 - ETA: 39s - loss: 16.0677 - acc: 0.003 - ETA: 40s - loss: 16.0723 - acc: 0.002 - ETA: 39s - loss: 16.0761 - acc: 0.002 - ETA: 39s - loss: 16.0406 - acc: 0.004 - ETA: 39s - loss: 16.0461 - acc: 0.004 - ETA: 39s - loss: 16.0509 - acc: 0.004 - ETA: 39s - loss: 16.0551 - acc: 0.003 - ETA: 39s - loss: 16.0588 - acc: 0.003 - ETA: 39s - loss: 16.0621 - acc: 0.003 - ETA: 39s - loss: 16.0651 - acc: 0.003 - ETA: 39s - loss: 16.0677 - acc: 0.003 - ETA: 39s - loss: 16.0461 - acc: 0.004 - ETA: 38s - loss: 16.0494 - acc: 0.004 - ETA: 38s - loss: 16.0524 - acc: 0.004 - ETA: 38s - lo

10000/10000 [==============================] - ETA: 14s - loss: 16.0395 - acc: 0.004 - ETA: 14s - loss: 16.0399 - acc: 0.004 - ETA: 14s - loss: 16.0402 - acc: 0.004 - ETA: 14s - loss: 16.0406 - acc: 0.004 - ETA: 13s - loss: 16.0410 - acc: 0.004 - ETA: 13s - loss: 16.0413 - acc: 0.004 - ETA: 13s - loss: 16.0417 - acc: 0.004 - ETA: 13s - loss: 16.0421 - acc: 0.004 - ETA: 13s - loss: 16.0401 - acc: 0.004 - ETA: 13s - loss: 16.0404 - acc: 0.004 - ETA: 13s - loss: 16.0408 - acc: 0.004 - ETA: 13s - loss: 16.0411 - acc: 0.004 - ETA: 12s - loss: 16.0415 - acc: 0.004 - ETA: 12s - loss: 16.0418 - acc: 0.004 - ETA: 12s - loss: 16.0422 - acc: 0.004 - ETA: 12s - loss: 16.0425 - acc: 0.004 - ETA: 12s - loss: 16.0429 - acc: 0.004 - ETA: 12s - loss: 16.0432 - acc: 0.004 - ETA: 12s - loss: 16.0436 - acc: 0.004 - ETA: 12s - loss: 16.0439 - acc: 0.004 - ETA: 11s - loss: 16.0442 - acc: 0.004 - ETA: 11s - loss: 16.0445 - acc: 0.004 - ETA: 11s - loss: 16.0449 - acc: 0.004 - ETA: 11s - loss: 16.0452 - acc: 0

 6528/10000 [==================>...........] - ETA: 43s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 43s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 41s - loss: 15.9502 - acc: 0.0104    - ETA: 42s - loss: 15.9922 - acc: 0.007 - ETA: 42s - loss: 16.0174 - acc: 0.006 - ETA: 42s - loss: 16.0341 - acc: 0.005 - ETA: 41s - loss: 16.0461 - acc: 0.004 - ETA: 41s - loss: 16.0551 - acc: 0.003 - ETA: 41s - loss: 16.0621 - acc: 0.003 - ETA: 41s - loss: 16.0677 - acc: 0.003 - ETA: 41s - loss: 16.0723 - acc: 0.002 - ETA: 41s - loss: 16.0761 - acc: 0.002 - ETA: 41s - loss: 16.0793 - acc: 0.002 - ETA: 40s - loss: 16.0821 - acc: 0.002 - ETA: 40s - loss: 16.0509 - acc: 0.004 - ETA: 40s - loss: 16.0551 - acc: 0.003 - ETA: 40s - loss: 16.0292 - acc: 0.005 - ETA: 40s - loss: 16.0341 - acc: 0.005 - ETA: 40s - loss: 16.0386 - acc: 0.004 - ETA: 40s - loss: 16.0425 - acc: 0.004 - ETA: 40s - loss: 15.9982 - acc: 0.007 - ETA: 39s - loss: 15.9807 - acc: 0.008 - ETA: 39s - loss: 15.9867 - acc: 0.008 - ETA: 39s - loss: 15.9

 6528/10000 [==================>...........] - ETA: 31s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 38s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 39s - loss: 16.1181 - acc: 0.0000e+0 - ETA: 40s - loss: 15.9922 - acc: 0.0078    - ETA: 39s - loss: 16.0174 - acc: 0.006 - ETA: 39s - loss: 15.9502 - acc: 0.010 - ETA: 40s - loss: 15.9742 - acc: 0.008 - ETA: 41s - loss: 15.9922 - acc: 0.007 - ETA: 40s - loss: 16.0062 - acc: 0.006 - ETA: 40s - loss: 16.0174 - acc: 0.006 - ETA: 39s - loss: 16.0265 - acc: 0.005 - ETA: 40s - loss: 16.0341 - acc: 0.005 - ETA: 40s - loss: 16.0019 - acc: 0.007 - ETA: 39s - loss: 16.0102 - acc: 0.006 - ETA: 39s - loss: 16.0174 - acc: 0.006 - ETA: 39s - loss: 16.0237 - acc: 0.005 - ETA: 39s - loss: 16.0292 - acc: 0.005 - ETA: 39s - loss: 16.0341 - acc: 0.005 - ETA: 39s - loss: 16.0386 - acc: 0.004 - ETA: 38s - loss: 16.0425 - acc: 0.004 - ETA: 38s - loss: 16.0222 - acc: 0.006 - ETA: 38s - loss: 16.0265 - acc: 0.005 - ETA: 38s - loss: 16.0305 - acc: 0.005 - ETA: 38s - loss: 

10000/10000 [==============================] - ETA: 14s - loss: 16.0616 - acc: 0.003 - ETA: 14s - loss: 16.0594 - acc: 0.003 - ETA: 14s - loss: 16.0597 - acc: 0.003 - ETA: 14s - loss: 16.0600 - acc: 0.003 - ETA: 13s - loss: 16.0603 - acc: 0.003 - ETA: 13s - loss: 16.0605 - acc: 0.003 - ETA: 13s - loss: 16.0608 - acc: 0.003 - ETA: 13s - loss: 16.0611 - acc: 0.003 - ETA: 13s - loss: 16.0613 - acc: 0.003 - ETA: 13s - loss: 16.0616 - acc: 0.003 - ETA: 13s - loss: 16.0619 - acc: 0.003 - ETA: 12s - loss: 16.0598 - acc: 0.003 - ETA: 12s - loss: 16.0601 - acc: 0.003 - ETA: 12s - loss: 16.0603 - acc: 0.003 - ETA: 12s - loss: 16.0606 - acc: 0.003 - ETA: 12s - loss: 16.0609 - acc: 0.003 - ETA: 12s - loss: 16.0611 - acc: 0.003 - ETA: 12s - loss: 16.0614 - acc: 0.003 - ETA: 12s - loss: 16.0616 - acc: 0.003 - ETA: 11s - loss: 16.0619 - acc: 0.003 - ETA: 11s - loss: 16.0621 - acc: 0.003 - ETA: 11s - loss: 16.0624 - acc: 0.003 - ETA: 11s - loss: 16.0582 - acc: 0.003 - ETA: 11s - loss: 16.0584 - acc: 0

In [19]:
scores=model.evaluate(image_data, y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))

10000/10000 [==============================] - ETA: 14 - ETA: 15 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ET

In [21]:
scores[1]*100

0.4